# LangChain: Agents

The core idea of agents is to use an LLM to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code).  
In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.


## Outline:

* Using built in LangChain tools: Python REPL, Pandas, Wikipedia...
* Defining your own tools

In [0]:
import os
import glob
from pathlib import Path
import pandas as pd

import openai
import langchain

#from langchain.llms import OpenAI
from langchain.llms import AzureOpenAI

#from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI

from langchain.schema import HumanMessage

from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

from langchain.chains import PALChain
from langchain.chains import LLMMathChain

from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

#### Loading the ´gpt-35-turbo´ model

In [0]:
openai.api_type = "azure"
openai.api_base = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
openai.api_key = os.environ["OPENAI_API_KEY"]

openai_model_name = "gpt-35-turbo"
openai_deploy_name = "model-gpt-35-turbo"
openai.api_version = "2023-07-01-preview"

In [0]:
llm = AzureChatOpenAI(openai_api_base=openai.api_base,
                      openai_api_version=openai.api_version,
                      deployment_name=openai_deploy_name,
                      openai_api_key=os.environ["OPENAI_API_KEY"],
                      openai_api_type=openai.api_type,
                      temperature=0,
                      max_retries=2
                      #max_tokens=4000,
                      )


llm


## Simple calls to Agents

PAL stands for Programme Aided Language Model. PALChain reads complex math problems (described in natural language) and generates programs (for solving the math problem) as the intermediate reasoning steps, but offloads the solution step to a runtime such as a Python interpreter. 

In this first call to the Python Agent, the LLM gets right anyway

In [0]:
query = "If my age is half of my dad's age and he is going to be 60 next year, what is the age of my brother, which is two years younger than me?"
response = llm(messages=[HumanMessage(content=query)])
print(response.content)
print(response)

In [0]:
palchain = PALChain.from_math_prompt(llm=llm, verbose=True)
palchain.run("If my age is half of my dad's age and he is going to be 60 next year, what is the age of my brother, which is two years younger than me?")

We can check the full prompt generated by palchain, with some few shots as examples:

In [0]:
palchain.prompt.template

But in this second call, the Python agent figures out how to provide the missing information (the year)

In [0]:
query = "In which day of week the Austrian National Holiday will be next year?"
response = llm(messages=[HumanMessage(content=query)])
print(response.content)
print(response)

In [0]:
palchain = PALChain.from_math_prompt(llm=llm, verbose=True)
palchain.run("In which day of week the Austrian National Holiday will be next year?")

## Using the Python Agent explicitly:

In [0]:
agent = create_python_agent(llm,
                            tool=PythonREPLTool(),
                            verbose=True
                            )

In [0]:
customer_list = [["Renato", "Fiacchini"], 
                 ["Charles", "Aznavour"],
                 ["Bernhard", "Thomas"],
                 ["Joel", "Billy"], 
                 ["David","Goliah"], 
                 ["Brani","Acer"],
                 ["Georg","Curious"]
                ]

In [0]:
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

In [0]:
agent.run(f"""What are the char codes for each name on the list in hexadecimal: {customer_list}""")

##### We have to be aware of the immense capacity of the models...

In [0]:
def collatz(n):
    print(n)
    while n != 1:
        if n%2 == 0:
            n /= 2
        else:
            n = (3*n) + 1
        print(int(n), end=" ")

collatz(45)

In [0]:
query1 = "Print the Collatz sequence for the number 45"
query2 = "What is the biggest prime number below 1 billion?"
response = llm(messages=[HumanMessage(content=query1)])
print(response.content)

In [0]:
langchain.debug=True
print(agent.run(query1))

## Loading specific Agents

In [0]:
!pip install -Uq wikipedia

In [0]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [0]:
agent= initialize_agent(tools,
                        llm,
                        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                        handle_parsing_errors=True,
                        verbose = True)

## Wikipedia (blocked by firewall)

In [0]:
import wikipedia
wikipedia.search("Barack")

In [0]:
ny = wikipedia.page("New York")
print(ny.title)
print(ny.url)
print(ny.content)
print(ny.links[0])

In [0]:
langchain.debug=True
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)
langchain.debug=False

## LLM-Math

In [0]:
langchain.debug=False
agent("How much is 25% of 300?")
langchain.debug=False

## Duck Duck Go (blocked by firewall)

In [0]:
!pip install -q duckduckgo-search

In [0]:
from langchain.tools import DuckDuckGoSearchRun

In [0]:
search = DuckDuckGoSearchRun()

In [0]:
search.run("Obama's first name?")

## Pandas Agent

In [0]:
df = pd.read_csv('./Data/Data.csv')
df.head(10)

In [0]:
langchain.debug=False
pandas_agent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=False)

In [0]:
pandas_agent.run("how many rows are there?")

In [0]:
pandas_agent.run("Is there a review about pillows? Please show me it")

## Define your own tool

In [0]:
from langchain.agents import tool
from datetime import date

In [0]:
@tool
def my_color(text: str) -> str:
    """I use different colors of T-shirts
    depending on the day of month.
    I use White in the even days and
    I use Black in the odd days
    This function tells me which color 
    to use"""
    day = date.today().day
    if day%2 == 0:
        return "White"
    return "Black"

In [0]:
agent= initialize_agent([my_color],
                        llm,
                        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                        handle_parsing_errors=True,
                        verbose = False)

In [0]:
langchain.debug=False

try:
    result = agent("What t-shirt color should I use today")
    print(result)
except: 
    print("exception on external access")

langchain.debug=False